In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import numpy as np
import pandas as pd
import scipy
from pandas.io.parsers import read_csv
from hybrid import *
from pylab import *
from collections import defaultdict
%matplotlib inline
rcParams['figure.figsize'] = (5, 5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[col].quantile(float(q)/Nlevel)
            df[col+'_g'+str(int(q))+'q'+str(threshold)] = (df[col] > threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)


In [16]:
# set parameters
Nrules = 5000
supp = 5
maxlen = 3
alpha = 0.001
beta = 0.15

# read data
path = '/Users/tongwang/Dropbox/Research/Projects/Published/Hybrid/github/'
Xtrain = read_csv(path + 'train.csv',header = 0, sep = ',')
Xtest = read_csv(path + 'test.csv',header = 0, sep = ',')
bbm = read_csv(path + 'rf_ybtrain.csv',header = 0,sep = ',')
Ybtrain = np.array(bbm['Yb'])
bbm = read_csv(path+'rf_ybtest.csv',header = 0,sep = ',')
Ybtest = np.array(bbm['Yb'])

ntrain = Xtrain.shape[0]
df = Xtrain.append(Xtest)
Y = np.array(df['Y'])
df.drop(['Y'], axis = 1, inplace = True) 
numericals = [col for col in df.columns if '_' not in col]
code_continuous(df,numericals, 5)

# define a hybrid model
model = hyb(df.iloc[:ntrain],Y[:ntrain],Ybtrain,alpha,beta)
# generate candidate rules
model.generate_rulespace(supp,maxlen,Nrules,method = 'randomforest')
# train the model
model.train(100) # 100 iterations. Could be incerased or decreased


yhat,covered_index = model.predict(df.iloc[ntrain:df.shape[0]], Y[ntrain:df.shape[0]],Ybtest)
test_index = list(range(ntrain,df.shape[0],1))
train_index = list(range(ntrain))

test_accuracy = sum(yhat==Y[test_index])/len(test_index)
test_explainability=len(covered_index)/len(test_index)
positive_rules = [model.prules[i] for i in model.positive_rule_set]
negative_rules = [model.nrules[i] for i in model.negative_rule_set]


Using random forest to generate rules ...


KeyboardInterrupt: 